# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [20]:
import pandas as pd
import requests

# 1) seu código aqui
df = pd.read_csv('SINASC_RO_2019.csv')
total_registros = df.shape[0]
print(total_registros)
registro_unico = df.drop_duplicates().shape[0]
# Não há duplicados
df.count()
if (total_registros-registro_unico == 0 ):
    print('Não há linha duplicada')
else:
    print('Há linha duplicada')

27028
Não há linha duplicada


In [25]:
# 2) seu código aqui
valores_missing = df.isnull().sum()
valores_missing
#percentual = valores_missing/total_registros
#percentual

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [30]:
# 3) seu código aqui
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

df_selecionado = df[colunas_interesse]

df_selecionado.head(10)
valores_missing_selecionado = df_selecionado.isnull().sum()
valores_missing_selecionado

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [100]:
# 4) seu código aqui
apgar_total = df_selecionado['APGAR5'].shape[0]
print(apgar_total)
apgar_limpo = df_selecionado.dropna(subset=['APGAR5'])
apgar_limpo_total = apgar_limpo.shape[0]
print(apgar_limpo_total)
colunas_interesse = apgar_limpo.isnull().sum()
print(colunas_interesse)

27028
26925
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [103]:
# 5) seu código aqui
df_selecionado[['ESTCIVMAE', 'CONSULTAS']] = df_selecionado[['ESTCIVMAE', 'CONSULTAS']].fillna(9)
print(df_selecionado)
total_sem_branco = df_selecionado.isnull().sum()
total_sem_branco

       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única          4    10.0  
1      37 a 41 semanas    Única          4     9.0  
2      37 a 41 semanas  

C:\Users\lukas\AppData\Local\Temp\ipykernel_5444\2057709982.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado[['ESTCIVMAE', 'CONSULTAS']] = df_selecionado[['ESTCIVMAE', 'CONSULTAS']].fillna(9)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         312
QTDFILVIVO       0
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [94]:
# 6) Seu código aqui


0        False
1        False
2        False
3        False
4        False
         ...  
27023    False
27024    False
27025    False
27026    False
27027    False
Name: QTDFILVIVO, Length: 27028, dtype: bool

In [102]:
# 7) seu código aqui
df_selecionado['QTDFILVIVO'] = df_selecionado['QTDFILVIVO'].fillna(0)
verificacao_fil_vivo = df_selecionado['QTDFILVIVO'].isnull().sum()
print(verificacao_fil_vivo)
df_selecionado.isnull().sum()

0


C:\Users\lukas\AppData\Local\Temp\ipykernel_5444\3630497396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado['QTDFILVIVO'] = df_selecionado['QTDFILVIVO'].fillna(0)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         312
QTDFILVIVO       0
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [105]:
# 8) seu código aqui
df_selecionado['ASFIXIA'] = df_selecionado['APGAR5'].apply(lambda x: 
    'normal' if x >= 8 else
    'asfixia leve' if x >= 6 else
    'asfixia moderada' if x >= 4 else
    'asfixia severa'
)

df_selecionado[['APGAR5', 'ASFIXIA']].head()


C:\Users\lukas\AppData\Local\Temp\ipykernel_5444\451288525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado['ASFIXIA'] = df_selecionado['APGAR5'].apply(lambda x:


,APGAR5,ASFIXIA
0,10.0,normal
1,9.0,normal
2,10.0,normal
3,10.0,normal
4,10.0,normal


In [106]:
# 9) seu código aqui
df_selecionado.rename(columns={
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5',
    'ASFIXIA': 'asfixia'  # Se já tivermos criado essa coluna antes
}, inplace=True)

# Exibir as colunas renomeadas
print(df_selecionado.columns)


Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar5', 'asfixia'],
      dtype='object')


C:\Users\lukas\AppData\Local\Temp\ipykernel_5444\31609961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado.rename(columns={


In [112]:
df_selecionado

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,asfixia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal


,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5,asfixia
418,1,14,5.0,4 a 7 anos,0.0,37 a 41 semanas,Única,3,5.0,asfixia moderada
1118,1,19,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,5.0,asfixia moderada
1282,1,23,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,5.0,asfixia moderada
2142,1,34,1.0,8 a 11 anos,7.0,37 a 41 semanas,Única,4,5.0,asfixia moderada
3429,1,25,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,5.0,asfixia moderada
3799,1,30,2.0,12 anos ou mais,0.0,32 a 36 semanas,Única,3,5.0,asfixia moderada
4093,1,42,5.0,4 a 7 anos,4.0,NaN,Única,2,5.0,asfixia moderada
4395,1,21,5.0,4 a 7 anos,3.0,37 a 41 semanas,Única,4,5.0,asfixia moderada
4456,1,22,5.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,5.0,asfixia moderada
5575,1,15,1.0,8 a 11 anos,0.0,NaN,Única,1,5.0,asfixia moderada
